In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report
from pyESNModified import ESN  # 导入修改后的ESN实现

# 数据预处理
def preprocess_data():
    train_df = pd.read_csv('UNSW_NB15_training-set.csv')
    test_df = pd.read_csv('UNSW_NB15_testing-set.csv')

    categorical_features = ['proto', 'service', 'state']
    numeric_features = train_df.columns.difference(categorical_features + ['label', 'attack_cat'])

    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

    # Modify these lines in ESNcolab.py
    X_train = preprocessor.fit_transform(train_df).toarray().astype(np.float64)
    y_train = train_df['label'].values.reshape(-1, 1).astype(np.float64)
    X_test = preprocessor.transform(test_df).toarray().astype(np.float64)
    y_test = test_df['label'].values.reshape(-1, 1).astype(np.float64)

    return X_train, y_train, X_test, y_test

def main():
    # 数据预处理
    X_train, y_train, X_test, y_test = preprocess_data()

    # 初始化带贪心捆绑的ESN
    esn = ESN(
        n_inputs=X_train.shape[1],
        n_outputs=1,
        n_reservoir=500,
        spectral_radius=0.95,
        sparsity=0.2,
        noise=0.001,
        teacher_scaling=1.0,
        teacher_forcing=True,
        random_state=42,
        use_bundling=True,  # 启用贪心捆绑
        K=3                # 设置最大冲突数
    )

    # 训练和预测
    esn.fit(X_train, y_train)
    pred_test = esn.predict(X_test)
    y_pred = (pred_test > 0.5).astype(int)

    # 评估结果
    print("Greedy Bundling ESN Performance:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred))

if __name__ == "__main__":
    main()